In [51]:
import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass
from config import api_key

@dataclass
class Holiday:
    name: str
    date: datetime
    
    def __str__(self):
        return f"{self.name} ({self.date.isoformat()})"

    def __gt__(self, other):
        return self.date > other.date
    
    def __ge__(self, other):
        return self.date >= other.date

    def __eq__(self, other):
        return self.date == other.date

class Holiday_List:
    def __init__(self):
       self.inner_holidays = []

    def find_holiday(self, holiday_name, date):
        # Find Holiday instance in inner_holidays
        index = self.inner_holidays.index(Holiday(holiday_name, date.datetime()))
        # Return Holiday instance
        return self.inner_holidays[index]

    def remove_holiday(self, index):
        # Find the Holiday instance's index in inner_holidays by passing find_holiday as a parameter
        # Remove the Holiday instance from inner_holidays
        self.inner_holidays.remove(index)
        # Inform user you deleted the holiday
        print(f"\n{index} has been removed from the list.\n")

    def add_holiday(self, holiday_obj):
        # Make sure holidayObj is an Holiday Object by checking the type
        if type(holiday_obj) == Holiday:
            # Use inner_holidays.append(holidayObj) to add holiday
            self.inner_holidays.append(holiday_obj)
            # print to the user that you added a holiday
            print(f"\n{holiday_obj} has been added to the list.\n")
        else:
            print(f"\nInvalid parameter, {holiday_obj} is not a Holiday object.\n")

    def scrape_holidays(self):
        # Scrape Holidays from https://www.timeanddate.com/holidays/us/ 
        months = {"Jan": "1", "Feb": "2", "Mar": "3", "Apr": "4", "May": "5", "Jun": "6", 
                    "Jul": "7", "Aug": "8", "Sep": "9", "Oct": "10", "Nov": "11", "Dec": "12"}
        for year in range(2020, 2025):
            url = f"https://www.timeanddate.com/holidays/us/{year}"
            res = requests.get(url)
            soup = BeautifulSoup(res.text, "html.parser")
            table = soup.find("section", attrs={"class":"table-data__table"}).find("tbody")
            rows = table.find_all("tr")
            for i in range(1, len(rows)):
                try:
                    name = rows[i].find("a").text
                    scraped_date = rows[i].find("th").text
                    space = scraped_date.find(" ")
                    scraped_month = scraped_date[:space]
                    month = scraped_month.replace(scraped_month, months[scraped_month])
                    day = scraped_date[space+1:]
                    date = datetime.date(year, int(month), int(day))
                    holiday = Holiday(name, date)
                    # Check to see if name and date of holiday is in inner_holidays array
                    if holiday not in self.inner_holidays:
                        # Add non-duplicates to inner_holidays
                        self.add_holiday(holiday)
                # Handle any exceptions
                except AttributeError:
                    pass

    def filter_week(self, year, week_num):
        hols_in_year = list(filter(lambda hol: hol.date.isocalendar().year == year, self.inner_holidays))
        # Use a Lambda function to filter by week number and save this as holidays, use the filter on inner_holidays
        hols_in_week = filter(lambda hol: hol.date.isocalendar().week == week_num, hols_in_year)
        # Cast filter results as list
        holidays = list(hols_in_week)
        # return your holidays
        return holidays

    def display_week(self, holiday_list):
        # Use your filter_week to get list of holidays within a week as a parameter
        # Output formated holidays in the week. 
        for holiday in range(len(holiday_list)):
            print(str(holiday_list[holiday]))

    def get_weather(self, year, week_num):
        # Convert week_num to range between two days
        week = f"{year}-W{week_num}"
        week_mon = datetime.datetime.strptime(week + '-1', "%Y-W%W-%w").date().__str__()
        week_sun = datetime.datetime.strptime(week + '-0', "%Y-W%W-%w").date().__str__()
        # Use Try / Except to catch problems
        # Query API for weather in that week range
        url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Milwaukee/{week_mon}/{week_sun}?unitGroup=us&key={api_key}&contentType=json"
        json = requests.get(url).json()
        days = json['days']
        weather = {}
        for day in days:
            date = day['datetime']
            cond = day['conditions']
            weather[date] = cond
        # Format weather information and return weather list
        return weather

    def viewCurrentWeek(self):
        # Use the Datetime Module to look up current week and year
        today = datetime.date.today()
        curr_year = today.year
        curr_week = today.isocalendar().week
        # Use your filter_week function to get the list of holidays for the current week/year
        filt_hol = self.filter_week(curr_year, curr_week)
        # Ask user if they want to get the weather
        weather_prompt = input("Would you like to see this week's weather? [y/n]: ")
        print("\nThese are the holidays for this week: ")
        valid = 'n'
        while valid == 'n':  
            if weather_prompt == 'n':
                valid = 'y'
                # Use your display_week function to display the holidays in the week
                self.display_week(filt_hol)
            elif weather_prompt == 'y':
                valid = 'y'
                # If yes, use your getWeather function and display results
                weather = self.get_weather(curr_year, curr_week)
                for hol in filt_hol:
                    print(f"{str(hol)} - {weather[hol.date.__str__()]}")
            else:
                print("Invalid input, expecting 'y' or 'n'.")

In [52]:
hol_list = Holiday_List()
hol_list.scrape_holidays()


New Year's Day (2020-01-01) has been added to the list.


World Braille Day (2020-01-04) has been added to the list.


Epiphany (2020-01-06) has been added to the list.


Asarah B'Tevet (2020-01-07) has been added to the list.


Battle of New Orleans (2020-01-08) has been added to the list.


Stephen Foster Memorial Day (2020-01-13) has been added to the list.


Orthodox New Year (2020-01-14) has been added to the list.


Lee-Jackson Day (2020-01-17) has been added to the list.


World Religion Day (2020-01-19) has been added to the list.


Martin Luther King Jr. Day (2020-01-20) has been added to the list.


International Day of Education (2020-01-24) has been added to the list.


Lunar New Year (2020-01-25) has been added to the list.


International Customs Day (2020-01-26) has been added to the list.


International Day of Commemoration in Memory of the Victims of the Holocaust (2020-01-27) has been added to the list.


Kansas Day (2020-01-29) has been added to the list.


Nationa

In [80]:
hol_list.inner_holidays

[Holiday(name="New Year's Day", date=datetime.date(2020, 1, 1)),
 Holiday(name='World Braille Day', date=datetime.date(2020, 1, 4)),
 Holiday(name='Epiphany', date=datetime.date(2020, 1, 6)),
 Holiday(name="Asarah B'Tevet", date=datetime.date(2020, 1, 7)),
 Holiday(name='Battle of New Orleans', date=datetime.date(2020, 1, 8)),
 Holiday(name='Stephen Foster Memorial Day', date=datetime.date(2020, 1, 13)),
 Holiday(name='Orthodox New Year', date=datetime.date(2020, 1, 14)),
 Holiday(name='Lee-Jackson Day', date=datetime.date(2020, 1, 17)),
 Holiday(name='World Religion Day', date=datetime.date(2020, 1, 19)),
 Holiday(name='Martin Luther King Jr. Day', date=datetime.date(2020, 1, 20)),
 Holiday(name='International Day of Education', date=datetime.date(2020, 1, 24)),
 Holiday(name='Lunar New Year', date=datetime.date(2020, 1, 25)),
 Holiday(name='International Customs Day', date=datetime.date(2020, 1, 26)),
 Holiday(name='International Day of Commemoration in Memory of the Victims of the H

In [79]:
hol_name = input("Holiday Name: ").strip()
filt_hol = list(filter(lambda hol: hol.name == hol_name, hol_list.inner_holidays))
print(filt_hol)
for hol in filt_hol:
    clean = str(hol)
    print(clean)
hol_year = int(input("Holiday Year: ").strip())
chosen_hol = list(filter(lambda hol: hol.date.isocalendar().year == hol_year, filt_hol))
chosen_hol
# index = hol_list.inner_holidays.index(filt_hol[0])
# index


[Holiday(name='International Day for the Elimination of Violence against Women', date=datetime.date(2020, 11, 25)), Holiday(name='International Day for the Elimination of Violence against Women', date=datetime.date(2021, 11, 25)), Holiday(name='International Day for the Elimination of Violence against Women', date=datetime.date(2022, 11, 25)), Holiday(name='International Day for the Elimination of Violence against Women', date=datetime.date(2023, 11, 25)), Holiday(name='International Day for the Elimination of Violence against Women', date=datetime.date(2024, 11, 25))]
International Day for the Elimination of Violence against Women (2020-11-25)
International Day for the Elimination of Violence against Women (2021-11-25)
International Day for the Elimination of Violence against Women (2022-11-25)
International Day for the Elimination of Violence against Women (2023-11-25)
International Day for the Elimination of Violence against Women (2024-11-25)


[Holiday(name='International Day for the Elimination of Violence against Women', date=datetime.date(2020, 11, 25))]